In [2]:
from sklearn import datasets
import math
import matplotlib.pyplot as plt
import numpy as np
import progressbar

from sklearn.datasets.openml import fetch_openml

from scratchkit.dl.optim import Adam
from scratchkit.dl.losses import CrossEntropy
from scratchkit.dl.layers import  Dense, Dropout, Flatten, Activation, Reshape, BatchNormalization
from scratchkit.dl import NeuralNetwork

In [3]:
class GAN():
    """A Generative Adversarial Network with deep fully-connected neural nets as
    Generator and Discriminator.
    Training Data: MNIST Handwritten Digits (28x28 images)
    """
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.img_dim = self.img_rows * self.img_cols
        self.latent_dim = 100

        optimizer = Adam(learning_rate=0.0002, b1=0.5)
        loss_function = CrossEntropy

        # Build the discriminator
        self.discriminator = self.build_discriminator(optimizer, loss_function)

        # Build the generator
        self.generator = self.build_generator(optimizer, loss_function)

        # Build the combined model
        self.combined = NeuralNetwork(optimizer=optimizer, loss=loss_function)
        self.combined.layers.extend(self.generator.layers)
        self.combined.layers.extend(self.discriminator.layers)

        print ()
        self.generator.summary(name="Generator")
        self.discriminator.summary(name="Discriminator")

    def build_generator(self, optimizer, loss_function):
        
        model = NeuralNetwork(optimizer=optimizer, loss=loss_function)

        model.add(Dense(256, input_shape=(self.latent_dim,)))
        model.add(Activation('leaky_relu'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(Activation('leaky_relu'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(Activation('leaky_relu'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.img_dim))
        model.add(Activation('tanh'))

        return model

    def build_discriminator(self, optimizer, loss_function):
        
        model = NeuralNetwork(optimizer=optimizer, loss=loss_function)

        model.add(Dense(512, input_shape=(self.img_dim,)))
        model.add(Activation('leaky_relu'))
        model.add(Dropout(0.5))
        model.add(Dense(256))
        model.add(Activation('leaky_relu'))
        model.add(Dropout(0.5))
        model.add(Dense(2))
        model.add(Activation('softmax'))

        return model

    def train(self, n_epochs, batch_size=128, save_interval=50):

        mnist = fetch_openml('mnist_784')

        X = mnist.data
        y = mnist.target

        # Rescale [-1, 1]
        X = (X.astype(np.float32) - 127.5) / 127.5

        half_batch = int(batch_size / 2)

        for epoch in range(n_epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            self.discriminator.set_trainable(True)

            # Select a random half batch of images
            idx = np.random.randint(0, X.shape[0], half_batch)
            imgs = X[idx]

            # Sample noise to use as generator input
            noise = np.random.normal(0, 1, (half_batch, self.latent_dim))

            # Generate a half batch of images
            gen_imgs = self.generator.predict(noise)

            # Valid = [1, 0], Fake = [0, 1]
            valid = np.concatenate((np.ones((half_batch, 1)), np.zeros((half_batch, 1))), axis=1)
            fake = np.concatenate((np.zeros((half_batch, 1)), np.ones((half_batch, 1))), axis=1)

            # Train the discriminator
            d_loss_real, d_acc_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake, d_acc_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * (d_loss_real + d_loss_fake)
            d_acc = 0.5 * (d_acc_real + d_acc_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            # We only want to train the generator for the combined model
            self.discriminator.set_trainable(False)

            # Sample noise and use as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # The generator wants the discriminator to label the generated samples as valid
            valid = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))), axis=1)

            # Train the generator
            g_loss, g_acc = self.combined.train_on_batch(noise, valid)

            # Display the progress
            print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, acc: %.2f%%]" % (epoch, d_loss, 100*d_acc, g_loss, 100*g_acc))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5 # Grid size
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        # Generate images and reshape to image shape
        gen_imgs = self.generator.predict(noise).reshape((-1, self.img_rows, self.img_cols))

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        plt.suptitle("Generative Adversarial Network")
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("mnist_%d.png" % epoch)
        plt.close()


In [4]:
gan = GAN()
gan.train(n_epochs=200000, batch_size=64, save_interval=400)


+-----------+
| Generator |
+-----------+
Input Shape: (100,)
+------------------------+------------+--------------+
| Layer Type             | Parameters | Output Shape |
+------------------------+------------+--------------+
| Dense                  | 25856      | (256,)       |
| Activation (LeakyReLU) | 0          | (256,)       |
| BatchNormalization     | 512        | (256,)       |
| Dense                  | 131584     | (512,)       |
| Activation (LeakyReLU) | 0          | (512,)       |
| BatchNormalization     | 1024       | (512,)       |
| Dense                  | 525312     | (1024,)      |
| Activation (LeakyReLU) | 0          | (1024,)      |
| BatchNormalization     | 2048       | (1024,)      |
| Dense                  | 803600     | (784,)       |
| Activation (TanH)      | 0          | (784,)       |
+------------------------+------------+--------------+
Total Parameters: 1489936

+---------------+
| Discriminator |
+---------------+
Input Shape: (784,)
+----------

98 [D loss: 0.347099, acc: 78.12%] [G loss: 0.767181, acc: 29.69%]
99 [D loss: 0.332385, acc: 84.38%] [G loss: 0.764070, acc: 31.25%]
100 [D loss: 0.333894, acc: 85.94%] [G loss: 0.760786, acc: 34.38%]
101 [D loss: 0.346845, acc: 76.56%] [G loss: 0.772262, acc: 31.25%]
102 [D loss: 0.326810, acc: 85.94%] [G loss: 0.765402, acc: 39.06%]
103 [D loss: 0.340281, acc: 81.25%] [G loss: 0.773974, acc: 32.81%]
104 [D loss: 0.323360, acc: 82.81%] [G loss: 0.768833, acc: 28.12%]
105 [D loss: 0.325561, acc: 84.38%] [G loss: 0.774862, acc: 28.12%]
106 [D loss: 0.315384, acc: 87.50%] [G loss: 0.784948, acc: 23.44%]
107 [D loss: 0.340637, acc: 84.38%] [G loss: 0.777600, acc: 29.69%]
108 [D loss: 0.316002, acc: 90.62%] [G loss: 0.785908, acc: 28.12%]
109 [D loss: 0.318188, acc: 90.62%] [G loss: 0.780762, acc: 31.25%]
110 [D loss: 0.334284, acc: 87.50%] [G loss: 0.772272, acc: 25.00%]
111 [D loss: 0.336129, acc: 82.81%] [G loss: 0.780764, acc: 20.31%]
112 [D loss: 0.334135, acc: 82.81%] [G loss: 0.787

220 [D loss: 0.280346, acc: 98.44%] [G loss: 0.858757, acc: 7.81%]
221 [D loss: 0.278618, acc: 93.75%] [G loss: 0.857458, acc: 3.12%]
222 [D loss: 0.292527, acc: 96.88%] [G loss: 0.872587, acc: 6.25%]
223 [D loss: 0.291277, acc: 96.88%] [G loss: 0.870116, acc: 7.81%]
224 [D loss: 0.295159, acc: 95.31%] [G loss: 0.860812, acc: 7.81%]
225 [D loss: 0.282933, acc: 96.88%] [G loss: 0.872952, acc: 4.69%]
226 [D loss: 0.287515, acc: 95.31%] [G loss: 0.871702, acc: 10.94%]
227 [D loss: 0.294642, acc: 96.88%] [G loss: 0.872660, acc: 9.38%]
228 [D loss: 0.279169, acc: 100.00%] [G loss: 0.865257, acc: 7.81%]
229 [D loss: 0.284982, acc: 95.31%] [G loss: 0.862374, acc: 7.81%]
230 [D loss: 0.286092, acc: 95.31%] [G loss: 0.862349, acc: 9.38%]
231 [D loss: 0.274052, acc: 100.00%] [G loss: 0.853093, acc: 12.50%]
232 [D loss: 0.279469, acc: 96.88%] [G loss: 0.864645, acc: 6.25%]
233 [D loss: 0.274021, acc: 98.44%] [G loss: 0.860760, acc: 7.81%]
234 [D loss: 0.284070, acc: 100.00%] [G loss: 0.866961, ac

342 [D loss: 0.277469, acc: 96.88%] [G loss: 0.919654, acc: 3.12%]
343 [D loss: 0.278779, acc: 96.88%] [G loss: 0.900942, acc: 6.25%]
344 [D loss: 0.274907, acc: 98.44%] [G loss: 0.916747, acc: 6.25%]
345 [D loss: 0.274084, acc: 100.00%] [G loss: 0.905225, acc: 3.12%]
346 [D loss: 0.279263, acc: 96.88%] [G loss: 0.925315, acc: 4.69%]
347 [D loss: 0.281316, acc: 93.75%] [G loss: 0.903552, acc: 6.25%]
348 [D loss: 0.273061, acc: 96.88%] [G loss: 0.926084, acc: 1.56%]
349 [D loss: 0.277502, acc: 100.00%] [G loss: 0.915371, acc: 0.00%]
350 [D loss: 0.273890, acc: 98.44%] [G loss: 0.911996, acc: 3.12%]
351 [D loss: 0.282572, acc: 98.44%] [G loss: 0.918395, acc: 7.81%]
352 [D loss: 0.271737, acc: 98.44%] [G loss: 0.923773, acc: 3.12%]
353 [D loss: 0.258294, acc: 100.00%] [G loss: 0.914146, acc: 1.56%]
354 [D loss: 0.266970, acc: 100.00%] [G loss: 0.929735, acc: 1.56%]
355 [D loss: 0.280045, acc: 98.44%] [G loss: 0.929107, acc: 6.25%]
356 [D loss: 0.259767, acc: 98.44%] [G loss: 0.926808, acc

464 [D loss: 0.258730, acc: 98.44%] [G loss: 0.989623, acc: 1.56%]
465 [D loss: 0.259104, acc: 98.44%] [G loss: 0.975762, acc: 1.56%]
466 [D loss: 0.273032, acc: 95.31%] [G loss: 0.981730, acc: 1.56%]
467 [D loss: 0.253486, acc: 100.00%] [G loss: 0.970180, acc: 1.56%]
468 [D loss: 0.258578, acc: 96.88%] [G loss: 0.963117, acc: 6.25%]
469 [D loss: 0.278079, acc: 96.88%] [G loss: 0.988358, acc: 4.69%]
470 [D loss: 0.255946, acc: 100.00%] [G loss: 0.987030, acc: 0.00%]
471 [D loss: 0.249628, acc: 100.00%] [G loss: 0.967112, acc: 4.69%]
472 [D loss: 0.251600, acc: 98.44%] [G loss: 0.996908, acc: 1.56%]
473 [D loss: 0.247120, acc: 98.44%] [G loss: 0.995354, acc: 3.12%]
474 [D loss: 0.249325, acc: 100.00%] [G loss: 0.975919, acc: 3.12%]
475 [D loss: 0.266208, acc: 93.75%] [G loss: 0.991841, acc: 1.56%]
476 [D loss: 0.250054, acc: 98.44%] [G loss: 0.999499, acc: 0.00%]
477 [D loss: 0.240325, acc: 100.00%] [G loss: 0.989045, acc: 3.12%]
478 [D loss: 0.257337, acc: 96.88%] [G loss: 0.995204, ac

...

14843 [D loss: 0.689699, acc: 50.00%] [G loss: 0.690129, acc: 53.12%]
14844 [D loss: 0.719977, acc: 37.50%] [G loss: 0.707288, acc: 46.88%]
14845 [D loss: 0.699497, acc: 51.56%] [G loss: 0.728393, acc: 42.19%]
14846 [D loss: 0.689937, acc: 50.00%] [G loss: 0.674666, acc: 62.50%]
14847 [D loss: 0.690065, acc: 56.25%] [G loss: 0.698609, acc: 50.00%]
14848 [D loss: 0.689861, acc: 51.56%] [G loss: 0.690754, acc: 54.69%]
14849 [D loss: 0.701854, acc: 48.44%] [G loss: 0.696958, acc: 51.56%]
14850 [D loss: 0.694984, acc: 53.12%] [G loss: 0.717019, acc: 39.06%]
14851 [D loss: 0.677430, acc: 59.38%] [G loss: 0.678501, acc: 57.81%]
14852 [D loss: 0.690761, acc: 51.56%] [G loss: 0.701847, acc: 53.12%]
14853 [D loss: 0.703544, acc: 50.00%] [G loss: 0.673557, acc: 56.25%]
14854 [D loss: 0.656401, acc: 68.75%] [G loss: 0.695416, acc: 53.12%]
14855 [D loss: 0.693088, acc: 48.44%] [G loss: 0.695158, acc: 48.44%]
14856 [D loss: 0.695983, acc: 53.12%] [G loss: 0.698597, acc: 42.19%]
14857 [D loss: 0.691

14961 [D loss: 0.704614, acc: 48.44%] [G loss: 0.698777, acc: 51.56%]
14962 [D loss: 0.703366, acc: 48.44%] [G loss: 0.725021, acc: 35.94%]
14963 [D loss: 0.678987, acc: 54.69%] [G loss: 0.715140, acc: 43.75%]
14964 [D loss: 0.672347, acc: 65.62%] [G loss: 0.698775, acc: 53.12%]
14965 [D loss: 0.689603, acc: 60.94%] [G loss: 0.726809, acc: 39.06%]
14966 [D loss: 0.704991, acc: 37.50%] [G loss: 0.709652, acc: 50.00%]
14967 [D loss: 0.702952, acc: 50.00%] [G loss: 0.679724, acc: 60.94%]
14968 [D loss: 0.690708, acc: 53.12%] [G loss: 0.698386, acc: 48.44%]
14969 [D loss: 0.690783, acc: 51.56%] [G loss: 0.708192, acc: 42.19%]
14970 [D loss: 0.697150, acc: 56.25%] [G loss: 0.702510, acc: 39.06%]
14971 [D loss: 0.705189, acc: 50.00%] [G loss: 0.701386, acc: 40.62%]
14972 [D loss: 0.668875, acc: 57.81%] [G loss: 0.701986, acc: 42.19%]
14973 [D loss: 0.672487, acc: 65.62%] [G loss: 0.695345, acc: 53.12%]
14974 [D loss: 0.692592, acc: 53.12%] [G loss: 0.698766, acc: 51.56%]
14975 [D loss: 0.681

15079 [D loss: 0.703634, acc: 48.44%] [G loss: 0.685853, acc: 62.50%]
15080 [D loss: 0.685254, acc: 53.12%] [G loss: 0.718679, acc: 42.19%]
15081 [D loss: 0.697075, acc: 51.56%] [G loss: 0.702738, acc: 51.56%]
15082 [D loss: 0.696286, acc: 50.00%] [G loss: 0.683262, acc: 62.50%]
15083 [D loss: 0.670447, acc: 54.69%] [G loss: 0.680986, acc: 54.69%]
15084 [D loss: 0.674491, acc: 62.50%] [G loss: 0.687402, acc: 51.56%]
15085 [D loss: 0.692645, acc: 42.19%] [G loss: 0.710108, acc: 45.31%]
15086 [D loss: 0.720958, acc: 40.62%] [G loss: 0.686436, acc: 48.44%]
15087 [D loss: 0.676165, acc: 57.81%] [G loss: 0.704816, acc: 46.88%]
15088 [D loss: 0.692406, acc: 54.69%] [G loss: 0.692497, acc: 48.44%]
15089 [D loss: 0.703978, acc: 48.44%] [G loss: 0.705919, acc: 43.75%]
15090 [D loss: 0.700314, acc: 51.56%] [G loss: 0.701589, acc: 40.62%]
15091 [D loss: 0.683293, acc: 56.25%] [G loss: 0.699665, acc: 46.88%]
15092 [D loss: 0.680448, acc: 67.19%] [G loss: 0.738940, acc: 35.94%]
15093 [D loss: 0.712

15197 [D loss: 0.700532, acc: 46.88%] [G loss: 0.710699, acc: 48.44%]
15198 [D loss: 0.685500, acc: 54.69%] [G loss: 0.708445, acc: 40.62%]
15199 [D loss: 0.706222, acc: 46.88%] [G loss: 0.702279, acc: 46.88%]
15200 [D loss: 0.683779, acc: 60.94%] [G loss: 0.718881, acc: 39.06%]
15201 [D loss: 0.701100, acc: 54.69%] [G loss: 0.724683, acc: 40.62%]
15202 [D loss: 0.715059, acc: 43.75%] [G loss: 0.683221, acc: 57.81%]
15203 [D loss: 0.696849, acc: 53.12%] [G loss: 0.684871, acc: 50.00%]
15204 [D loss: 0.704246, acc: 51.56%] [G loss: 0.692941, acc: 54.69%]
15205 [D loss: 0.703573, acc: 53.12%] [G loss: 0.706729, acc: 43.75%]
15206 [D loss: 0.689829, acc: 51.56%] [G loss: 0.710237, acc: 42.19%]
15207 [D loss: 0.696817, acc: 53.12%] [G loss: 0.708066, acc: 50.00%]
15208 [D loss: 0.724314, acc: 42.19%] [G loss: 0.716096, acc: 34.38%]
15209 [D loss: 0.694292, acc: 48.44%] [G loss: 0.682126, acc: 56.25%]
15210 [D loss: 0.699812, acc: 51.56%] [G loss: 0.694041, acc: 48.44%]
15211 [D loss: 0.726

KeyboardInterrupt: 